# Working with ECOSTRESS L2T LSTE Data  

**Summary** 

This notebook will show how to access [ECOsystem Spaceborne Thermal Radiometer Experiment on Space Station (ECOSTRESS)](https://ecostress.jpl.nasa.gov/) data programmatically using the [`earthaccess`](https://github.com/nsidc/earthaccess) python library to authenticate, search, download, and stream (directly access) data. It shows how to work with ECOSTRESS Tiled Land Surface Temperature and Emissivity ([`ECOSTRESS_L2T_LSTE`](https://doi.org/10.5067/ECOSTRESS/ECO_L2T_LSTE.002)) product hosted in the Earthdata Cloud and managed by the Land Processes Distributed Active Archive Center ([LP DAAC](https://lpdaac.usgs.gov/)). 

**Background**  

The [ECOSTRESS mission](https://ecostress.jpl.nasa.gov/) is answering these questions by accurately measuring the temperature of plants. Plants regulate their temperature by releasing water through tiny pores on their leaves called stomata. If they have sufficient water they can maintain their temperature, but if there is insufficient water, their temperatures rise and this temperature rise can be measured with ECOSTRESS. The images acquired by ECOSTRESS are the most detailed temperature images of the surface ever acquired from space and can be used to measure the temperature of an individual farm field. These temperature images, along with auxiliary inputs, are used to produce one of the primary science outputs of ECOSTRESS: evapotranspiration, an indicator of plant health via the measure of evaporation and transpiration of water through a plant.

More details about ECOSTRESS and its associated products can be found on the [ECOSTRESS website](https://ecostress.jpl.nasa.gov/) and [ECOSTRESS product pages](https://lpdaac.usgs.gov/product_search/?collections=ECOSTRESS&status=Operational&view=cards&sort=title) hosted by the Land Processes Distributed Active Archive Center (LP DAAC).

**Learning Objectives**  

- How to search ECOSTRESS data using `earthaccess`
- How to stream or download ECOSTRESS data
- How to clip ECOSTRESS data to a Region of Interest (ROI)
- How to quality filter ECOSTRESS data
- How to export the processed ECOSTRESS data

**Requirements**  

- NASA [Earthdata Login](https://urs.earthdata.nasa.gov/) account. If you do not have an Earthdata Account, you can create one [here](https://urs.earthdata.nasa.gov/users/new). 
- A compatible Python Environment - See **setup_instructions.md** in the `/setup/` folder 

**Tutorial Outline**

1. Setup
2. Searching for ECOSTRESS L2T LSTE Data
3. Accessing ECOSTRESS L2T LSTE Data
4. Clipping ECOSTRESS L2T LSTE Data to a Region of Interest (ROI)
5. Quality Filtering ECOSTRESS L2T LSTE Data
6. Exporting Processed ECOSTRESS L2T LSTE Data

## 1. Setup 

Import the required libraries.

In [ ]:
# Import Packages
import warnings
# Some cells may generate warnings that we can ignore. Comment below lines to see.
warnings.filterwarnings('ignore')

import os, sys, shutil
import earthaccess
import numpy as np
from osgeo import gdal
import rasterio as rio
import rioxarray as rxr
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import geopandas as gp
from zipfile import ZipFile 

### Authentication

Log into Earthdata using the `Auth` and `login` functions from the `earthaccess` library. The `persist=True` argument will create a local `.netrc` file if it doesn't exist, or add your login info to an existing `.netrc` file. If no Earthdata Login credentials are found in the `.netrc` you'll be prompted for them.


In [ ]:
auth = earthaccess.login(persist = True)
# are we authenticated?
print(auth.authenticated)

## 2. Searching for ECOSTRESS L2T LSTE Data

In this example, we will use the cloud-hosted `ECOSTRESS_L2T_LSTE` product but the searching process can be used with other EMIT or ECOSTRESS products, other collections, or different data providers, as well as across multiple catalogs with some modification.  The Land Surface Temperature and Emissivity values from ECOSTRESS Level 2 Tiled Land Surface Temperature (ECO_L2T_LSTE) are derived from five thermal infrared (TIR) bands using a physics-based Temperature and Emissivity Separation (TES) algorithm. This tiled data product uses a modified version of the Military Grid Reference System (MGRS) which divides Universal Transverse Mercator (UTM) zones into square tiles that are 109.8 km by 109.8 km with a 70 meter (m) spatial resolution. 

### Define Your Query Parameters

We can search for granules using attributes such as collection short name, collection ID, acquisition time, and spatial footprint.

##### Spatial region of interest (ROI)

For this example, our spatial region of interest (ROI) will be Boulder city boundary. when we are working with multi-feature ROI, we can use a bounding box with larger spatial extent including all the features. To do this, we will first open a geojson file containing our region of interest (ROI) then simplify it to a bounding box by getting the bounds and putting them into a tuple. We will use the `total_bounds` property to get the bounding box of our ROI, and add that to a python tuple, which is the expected data type for the bounding_box parameter `earthaccess` `search_data`.


**Note:** If our features are spread out spatially in the ROI, we can search for data available for each feature separately to avoid accessing a large volume of data we do not need.

Our ROI is stored as a .zip file. First we need to extract all the members of the zip into a specific location. 

In [ ]:
with ZipFile('../../data/City_of_Boulder_City_Limits.zip', 'r') as zObject: 
    zObject.extractall( path="../../data")

Below, the polygon is opened using `geopandas` library and Coordinate Reference System (crs) is printed. Our polygon is in geographic coordinate reference system (`EPSG:3857`).

In [ ]:
polygon = gp.read_file('../../data/City_of_Boulder_City_Limits.shp')
polygon.crs

Since this data is in **EPSG:3857**, we need to reproject to **EPSG:4326** to get latitude and longitude values for our search. We can use the `to_crs` method from `geopandas` to reproject our polygon.

In [ ]:
polygon_4326 = polygon.to_crs("EPSG:4326")

In [ ]:
polygon_4326.hvplot(tiles='ESRI', color='#d95f02',alpha=0.6, crs='EPSG:4326', frame_height=405, frame_width=720, fontscale=2) 

In [ ]:
bbox = tuple(list(polygon_4326.total_bounds))
bbox

Below, the  parameters including `provider`, `short_name`, `version`, `bounding_box`, `temporal`, and `count` are used for our query.  

In [ ]:
results = earthaccess.search_data(
    provider='LPCLOUD',
    short_name='ECO_L2T_LSTE',
    version='002',
    bounding_box=bbox,
    temporal=('2023-07-01','2023-08-01'),
    count=100
)

Next, get the downloadable links for LSTE and quality layers using `data_links()` method from `earthaccess`. 

In [ ]:
lst_links = [l for dl in results for l in dl.data_links() if 'LST.tif' in l]
lst_links

In [ ]:
qc_links = [l for dl in results for l in dl.data_links() if 'QC.tif' in l]
qc_links

Let's take a look at the ECOSTRESS tiled data file name:  

        Filename: **ECOv002_L2T_LSTE_28527_009_13TDE_20230718T081442_0710_01_LST.tif**   

        ECO : Sensor  
        v002 : Product Version  
        L2T : Processing Level and Type (T = Tile)  
        LSTE : Geophysical Parameter  
        28527 : Orbit Number  
        009 : Scene ID  
        13TDE : Military Grid Reference System (MGRS) Tile ID  
        20230718 : Date of Acquisition (YYYYMMDD)  
        T081442 : Time of Acquisition (HHMMSS) (in UTC)  
        0710 : Build ID of software that generated product, Major+Minor (2+2 digits)  
        01 : Product Iteration Number  
        LST : Layer/band Name (each layer is a separate file)  
        .tif : Data Format for Tile  


Looking at Military Grid Reference System (MGRS) Tile ID of the outputs, they all are all in UTM Zone 13. 

## 3. Accessing ECOSTRESS L2T Land Surface Temperature and Emissivity Data

ECOSTRESS data is stored in NASA's Earthdata Cloud and can be accessed in different ways.

- *Downloaded* - This has been available since the existance of the NASA DAAC. Users can use the data link(s) to download the data files to their local working environment. This can be done whether the user is working from a non-cloud or cloud environment.
- *Streamed* - Streaming is on-the-fly random reading of remote files, i.e. files not saved locally. The data accessed, however, must be able to be held in the workspaces' memory. This can be done whether the user is working from a non-cloud or cloud environment.
- *Accessed in-place (i.e., direct s3 access)* - This is only available for working environment deployed in AWS us-west-2.

In this example, we will show how to stream the data. For that, the gdal configuration is set and the one of our LSTE files is read into the workspace using `open_rasterio` from the `rioxarray` library. Since the file consists of only 1 layer, we can `squeeze` it, removing the `band` dimension.

In [ ]:
rio_env = rio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_MAX_RETRY=10,
                  GDAL_HTTP_RETRY_DELAY=0.5)
rio_env.__enter__()

In [ ]:
eco_lst_ds = rxr.open_rasterio(lst_links[14]).squeeze('band', drop=True)
eco_lst_ds

As mentioned, the ECOSTRESS product we are using here is tiled and the CRS is dependent on UTM zone. For this tile, we can look at the `spatial_ref` variable through the interactive object above to see details such as the well-known-text (WKT) representation of the CRS and other attributes. 
We are using `hvplot` for visualization here. For detailed information on available open-source Python tools and libraries for data visualization see <https://pyviz.org/>.

Now let's plot the data using `hvplot`. `reproject` function is applied only for the visualization. Make sure to specify the CRS argument within the `hvplot.image` function so the ESRI imagery RBG background tile aligns with our scene.  



In [ ]:
size_opts = dict(frame_height=405, frame_width=720, fontscale=2)

eco_lst_ds.rio.reproject('EPSG:4326').hvplot.image(x='x', y='y', **size_opts, 
                                                   cmap='inferno', tiles='ESRI', xlabel='Longitude', 
                                                   ylabel='Latitude', title='ECOSTRESS LST (K)', 
                                                   crs='EPSG:4326')*polygon_4326.hvplot(color='Green',alpha=0.5, 
                                                                                   crs='EPSG:4326', rasterize=True)

## 4. Cropping ECOSTRESS Data  

`clip` function from `rasterio` is used to mask data outside of our region of interest. Before clipping, we need to reproject our ROI to the projection of our dataset which is UTM zone 13N. 

In [ ]:
polygon

In [ ]:
eco_lst_ds.rio.crs

In [ ]:
polygon_reproj = polygon.to_crs(eco_lst_ds.rio.crs)

In [ ]:
polygon_reproj.crs

In [ ]:
eco_lst_roi = eco_lst_ds.rio.clip(polygon_reproj.geometry.values, polygon_reproj.crs, all_touched=True)

In [ ]:
eco_lst_roi.hvplot.image(
    geo=True,cmap='inferno',**size_opts, tiles='ESRI',alpha=0.8, 
    title='Cropped ECOSTRESS LST (K)', xlabel='Longitude',ylabel='Latitude', 
    crs='EPSG:32613', rasterize=True)

Next, we will repeate the same process for the associated quality layer.

In [ ]:
eco_qc_ds = rxr.open_rasterio(qc_links[14]).squeeze('band', drop=True)

eco_qc_roi = eco_qc_ds.rio.clip(polygon_reproj.geometry.values, polygon_reproj.crs, all_touched=True) # assign a different value to fill value 

eco_qc_roi.hvplot.image(
    geo=True, cmap='inferno',**size_opts, tiles='ESRI',alpha=0.8, 
    title='Cropped ECOSTRESS LST (K)', xlabel='Longitude',ylabel='Latitude', 
    crs='EPSG:32613', rasterize=True)

## 5. Quality Filtering

The quality values are 16 digits bit values with bits 0 and 1 being the mandatory QA flag that will be interpreted as:

        00 = Pixel produced, best quality
        01 = Pixel produced, nominal quality. Either one or more of the following conditions are met:  

                1. emissivity in both bands 4 and 5 < 0.95, i.e. possible cloud contamination  
                2. low transmissivity due to high water vapor loading (<0.4), check PWV values and error estimates  
                3. Pixel falls on missing scan line in bands 1&5, and filled using spatial neural net. Check error estimates  
                Recommend more detailed analysis of other QC information  
        10 = Pixel produced, but cloud detected  
        11 = Pixel not produced due to missing/bad data, user should check Data quality flag bits  

The detailed quality information is provided in  Table 3-5 in ECOSTRESS [Product Specification Document](https://lpdaac.usgs.gov/documents/380/ECO2_PSD_V1.pdf). 


Below, the unique quality values are extracted from the clipped data, and only the values showing good quality are kept. 

In [ ]:
quality_vals = np.unique(eco_qc_roi.values).tolist()
good_q = [q for q in quality_vals if np.binary_repr(q, width=16)[-2:] == '00']
good_q


`.where` method is used to filter the quality and keep only the LSTE values generated with the best quality.  

In [ ]:
eco_lst_roi_mask = eco_lst_roi.where(eco_qc_roi.isin(good_q))
eco_lst_roi_mask

In [ ]:
eco_lst_roi_mask.hvplot.image(
    geo=True,cmap='inferno',**size_opts, tiles='ESRI',alpha=0.9, 
    title='Quality Masked ECOSTRESS LST (K)', xlabel='Longitude',ylabel='Latitude', 
    crs='EPSG:32613', rasterize=True)

## 6. Writing Outputs 

We now have a ECOSTRESS scene that is clipped to our ROI with only good quality values. Finally, we can save this file locally. 

In [ ]:
out_name = f"../../data/{lst_links[14].split('/')[-1].split('.tif')[0]}_clipped.tif"

eco_lst_roi_mask.rio.to_raster(raster_path=out_name, driver='COG')

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://www.earthdata.nasa.gov/centers/lp-daac>   

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I. 